In [ ]:
Objetivo: carregar somente os cabeçalhos (nomes das colunas) dos quatro arquivos CSV informados e mostrar em tela, de forma transposta, um quadro onde cada arquivo aparece como uma coluna e os nomes de campo ficam verticalmente.
Nada além disso será feito.

In [2]:
# ===============================================
# ETAPA: LEITURA DE CABEÇALHOS E EXIBIÇÃO TRANSPOSTA
# ===============================================

import os
import pandas as pd

# Arquivos informados (use exatamente estes caminhos)
csv_paths = [
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv",
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv",
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_REFS.csv",
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\DELTA_PROXY_DIAGNOSTICS.csv",
]

def read_headers_only(path: str):
    """
    Lê apenas os cabeçalhos do CSV (sem carregar dados).
    Tenta primeiro utf-8-sig, depois latin1.
    Retorna lista de nomes de colunas.
    """
    # Verificação de existência
    if not os.path.exists(path):
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")

    encodings = ["utf-8-sig", "latin1"]
    last_err = None
    for enc in encodings:
        try:
            # nrows=0 garante leitura apenas do header
            df = pd.read_csv(path, nrows=0, encoding=enc)
            return list(df.columns)
        except Exception as e:
            last_err = e
    # Se todas as tentativas falharem, reapresenta o último erro
    raise last_err

# Monta um DataFrame onde cada coluna representa um arquivo
series_by_file = {}
for p in csv_paths:
    headers = read_headers_only(p)
    name = os.path.basename(p)
    # Series indexadas por posição, para fácil visualização vertical
    series_by_file[name] = pd.Series(headers, dtype="object")

# Concatena lado a lado, alinhando pelo índice (posição dos cabeçalhos)
headers_matrix = pd.concat(series_by_file, axis=1)

# Exibição: nomes de arquivos como cabeçalhos de coluna
pd.set_option("display.max_rows", None)      # mostra todos os cabeçalhos
pd.set_option("display.max_columns", None)   # mostra todas as colunas (arquivos)
pd.set_option("display.width", 0)

print("Matriz transposta de cabeçalhos (cada coluna é um arquivo; linhas = nomes de campos na ordem original):")
display(headers_matrix)


Matriz transposta de cabeçalhos (cada coluna é um arquivo; linhas = nomes de campos na ordem original):


,A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv,A1_SECONDARIES_FOR_PEDRA_MODEL.csv,A1_SECONDARIES_REFS.csv,DELTA_PROXY_DIAGNOSTICS.csv
0,Timestamp,timestamp,delta_proxy_ref,col
1,coal_flow_furnace_t_h,coal_flow_furnace_t_h,tau_densa_ref,group
2,total_air_flow_knm3_h,vazao_ar_total_knm3_h,tau_diluida_ref,nan_rate
3,total_paf_air_flow_knm3_h,vazao_ar_prim_total_knm3_h,tau_backpass_ref,spearman_tau
4,te_of_hot_pri_air_in_aph_outl_adegc,temp_hot_pri_air_in_preaq_ar_saida,v_proxy_total_ref,spearman_vtotal
5,tau_densa,tau_densa,v_proxy_primary_ref,spearman_vprimary
6,tau_diluida,tau_diluida,NaN,trend_tau
7,tau_backpass,tau_backpass,NaN,trend_vtotal
8,o2_excess_pct,o2_excess_pct,NaN,trend_vprimary
9,delta_proxy,delta_proxy,NaN,score


In [3]:
# ===============================================
# ETAPA: SALVAMENTO DO DATAFRAME DE CABEÇALHOS
# ===============================================

import os

# Caminho de saída
output_dir = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\analises_preliminares"
os.makedirs(output_dir, exist_ok=True)

# Arquivo final
output_path = os.path.join(output_dir, "matriz_cabecalhos.csv")

# Salvar DataFrame
headers_matrix.to_csv(output_path, index=True, encoding="utf-8-sig")

print(f"Arquivo salvo em: {output_path}")


Arquivo salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\analises_preliminares\matriz_cabecalhos.csv


In [4]:
# ===============================================
# ETAPA: VARREDURA DE CSVs (APENAS CABEÇALHOS)
# ===============================================
import os
import pandas as pd

# Diretório base do projeto
base_dir = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

# Pastas a inspecionar (relativas a base_dir)
folders_to_scan = ["data", "outputs"]

# Colunas-alvo
target_cols = ["Wr", "Wm", "Wr_ref", "Wm_ref", "Wr_idx", "Wm_idx"]

# Onde salvar o inventário
output_dir = os.path.join(base_dir, "outputs")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "inventario_wr_wm.csv")

# Coletar resultados
results = []

for folder in folders_to_scan:
    folder_path = os.path.join(base_dir, folder)
    if not os.path.exists(folder_path):
        continue
    for root, _, files in os.walk(folder_path):
        for f in files:
            if not f.lower().endswith(".csv"):
                continue
            full_path = os.path.join(root, f)

            # Tenta ler só o header com duas codificações comuns
            df = None
            for enc in ("utf-8-sig", "latin1"):
                try:
                    df = pd.read_csv(full_path, nrows=0, encoding=enc)
                    break
                except Exception:
                    df = None

            if df is None:
                # Não conseguiu ler o cabeçalho — ignora
                continue

            found = [c for c in target_cols if c in df.columns]
            if found:
                results.append({
                    "arquivo": full_path,
                    "colunas_encontradas": ", ".join(found)
                })

# Monta DataFrame do inventário
df_inventory = pd.DataFrame(results, columns=["arquivo", "colunas_encontradas"])

# Salva inventário
df_inventory.to_csv(output_path, index=False, encoding="utf-8-sig")

# Exibe um resumo em tela (não lê dados, só informa)
print(f"Arquivos com colunas-alvo encontrados: {len(df_inventory)}")
print(f"Inventário salvo em: {output_path}")
if not df_inventory.empty:
    print("\nPrévia (até 10 linhas):")
    print(df_inventory.head(10).to_string(index=False))


Arquivos com colunas-alvo encontrados: 0
Inventário salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\inventario_wr_wm.csv


In [5]:
# ============================================================
# CONSOLIDAÇÃO MÍNIMA – AJUSTE DE CAMINHOS (MVP, SEM CÁLCULOS)
# ============================================================
import os
import pandas as pd

# Bases de caminho
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"                    # leitura (já existente)
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"         # escrita (criar se não existir)

# Garante que o diretório de destino exista
os.makedirs(DST_BASE, exist_ok=True)

# Fontes preferenciais (em A1_LOCAL)
SECUNDARIOS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL.csv"),
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv"),
]
REFS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_REFS.csv"),
]

# Saída padrão (em A1_LOCAL_REFAZIMENTO)
OUT_DIR  = os.path.join(DST_BASE, "data", "derivadas")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PATH = os.path.join(OUT_DIR, "tabela_wr_wm_padrao.csv")

# Colunas alvo do padrão (sem cálculos; apenas preencher se existirem nas fontes)
PADRAO_COLS = [
    "timestamp","zona","componente",
    "flw_total_c_t_h",
    "total_air_flow_knm3_h","total_paf_air_flow_knm3_h",
    "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy","tau_densa","tau_diluida","tau_backpass","o2_excess_pct",
    "Wr","Wm","Wr_ref","Wm_ref","Wr_idx","Wm_idx",
]

# Mapeamentos de nomes observados (não inferimos nada novo)
NAME_MAP = {
    "timestamp": "timestamp",
    "Timestamp": "timestamp",
    "vazao_ar_total_knm3_h": "total_air_flow_knm3_h",
    "vazao_ar_prim_total_knm3_h": "total_paf_air_flow_knm3_h",
    "temp_hot_pri_air_in_preaq_ar_saida": "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy": "delta_proxy",
    "tau_densa": "tau_densa",
    "tau_diluida": "tau_diluida",
    "tau_backpass": "tau_backpass",
    "o2_excess_pct": "o2_excess_pct",
}

def read_csv_best_effort(path):
    for enc in ("utf-8-sig", "latin1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    raise FileNotFoundError(f"Falha ao ler: {path}")

def pick_existing(paths):
    for p in paths:
        if os.path.exists(p):
            return p
    return None

# 1) Seleciona fontes existentes em A1_LOCAL
src_sec = pick_existing(SECUNDARIOS_CANDIDATOS)
src_ref = pick_existing(REFS_CANDIDATOS)

if src_sec is None:
    raise FileNotFoundError(
        "Não encontrei nenhum dos arquivos de secundárias esperados em A1_LOCAL\\outputs\\pedra\\ "
        "(A1_SECONDARIES_FOR_PEDRA_MODEL*.csv)."
    )

# 2) Carrega secundárias e aplica renomeações conhecidas
df_sec = read_csv_best_effort(src_sec)
rename_map = {c: NAME_MAP[c] for c in df_sec.columns if c in NAME_MAP}
df_sec = df_sec.rename(columns=rename_map)

if "timestamp" not in df_sec.columns:
    raise RuntimeError("A coluna 'timestamp' não foi encontrada após renomeação.")

# 3) Carrega referências (se existir), sem cálculos
df_ref = None
if src_ref is not None:
    try:
        df_ref = read_csv_best_effort(src_ref)
        ref_rename = {c: NAME_MAP[c] for c in df_ref.columns if c in NAME_MAP}
        if ref_rename:
            df_ref = df_ref.rename(columns=ref_rename)
    except Exception:
        df_ref = None

# 4) Expandir por 'zona' a partir de tau_* (apenas reshape; sem cálculos)
zonas_cols = [c for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df_sec.columns]
if zonas_cols:
    id_cols = [c for c in df_sec.columns if c not in zonas_cols]
    df_long = df_sec.melt(
        id_vars=id_cols,
        value_vars=zonas_cols,
        var_name="zona",
        value_name="tau_val"
    )
else:
    df_long = df_sec.copy()
    df_long["zona"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 5) Componente ausente nas fontes → marcar como ausente (sem inferir)
df_long["componente"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 6) Merge leve com referências por timestamp (somente novas colunas)
if df_ref is not None and "timestamp" in df_ref.columns:
    cols_to_merge = [c for c in df_ref.columns if c != "timestamp" and c not in df_long.columns]
    if cols_to_merge:
        df_long = df_long.merge(df_ref[["timestamp"] + cols_to_merge], on="timestamp", how="left")

# 7) Garantir colunas do padrão (criar vazias quando ausentes)
for col in PADRAO_COLS:
    if col not in df_long.columns:
        df_long[col] = pd.NA

# 8) Reordenar e salvar em A1_LOCAL_REFAZIMENTO
df_final = df_long[PADRAO_COLS].copy()
df_final.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

print(f"[OK] Tabela consolidada (sem cálculos) salva em:\n{OUT_PATH}")
print(f"Linhas: {len(df_final):,} | Colunas: {len(df_final.columns)}")
print("Colunas com algum preenchimento detectado:")
print([c for c in PADRAO_COLS if df_final[c].notna().any()])


[OK] Tabela consolidada (sem cálculos) salva em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao.csv
Linhas: 35,271 | Colunas: 18
Colunas com algum preenchimento detectado:
['timestamp', 'zona', 'componente', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'o2_excess_pct']


In [6]:
# ============================================================
# CONSOLIDAÇÃO (CORREÇÃO): PRESERVAR tau_* E REPLICAR POR ZONA
# ============================================================
import os
import pandas as pd

# Bases de caminho
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"                    # leitura (já existente)
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"         # escrita (criar se não existir)
os.makedirs(DST_BASE, exist_ok=True)

# Fontes em A1_LOCAL
SECUNDARIOS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL.csv"),
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv"),
]
REFS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_REFS.csv"),
]

# Saída em A1_LOCAL_REFAZIMENTO
OUT_DIR  = os.path.join(DST_BASE, "data", "derivadas")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PATH = os.path.join(OUT_DIR, "tabela_wr_wm_padrao.csv")

# Padrão de colunas (sem cálculos)
PADRAO_COLS = [
    "timestamp","zona","componente",
    "flw_total_c_t_h",
    "total_air_flow_knm3_h","total_paf_air_flow_knm3_h",
    "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy","tau_densa","tau_diluida","tau_backpass","o2_excess_pct",
    "Wr","Wm","Wr_ref","Wm_ref","Wr_idx","Wm_idx",
]

# Mapeamentos de nomes observados
NAME_MAP = {
    # timestamp
    "timestamp": "timestamp",
    "Timestamp": "timestamp",

    # vazões de ar
    "vazao_ar_total_knm3_h": "total_air_flow_knm3_h",
    "total_air_flow_knm3_h": "total_air_flow_knm3_h",
    "vazao_ar_prim_total_knm3_h": "total_paf_air_flow_knm3_h",
    "total_paf_air_flow_knm3_h": "total_paf_air_flow_knm3_h",

    # temperatura ar primário (APH)
    "temp_hot_pri_air_in_preaq_ar_saida": "te_of_hot_pri_air_in_aph_outl_adegc",
    "te_of_hot_pri_air_in_aph_outl_adegc": "te_of_hot_pri_air_in_aph_outl_adegc",

    # proxies / temps / O2
    "delta_proxy": "delta_proxy",
    "tau_densa": "tau_densa",
    "tau_diluida": "tau_diluida",
    "tau_backpass": "tau_backpass",
    "o2_excess_pct": "o2_excess_pct",

    # carvão
    "coal_flow_furnace_t_h": "flw_total_c_t_h",
    "flw_total_c_t_h": "flw_total_c_t_h",
}

def read_csv_best_effort(path):
    for enc in ("utf-8-sig", "latin1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    raise FileNotFoundError(f"Falha ao ler: {path}")

def pick_existing(paths):
    for p in paths:
        if os.path.exists(p):
            return p
    return None

# 1) Selecionar fontes
src_sec = pick_existing(SECUNDARIOS_CANDIDATOS)
src_ref = pick_existing(REFS_CANDIDATOS)

if src_sec is None:
    raise FileNotFoundError(
        "Não encontrei arquivos de secundárias em A1_LOCAL\\outputs\\pedra\\ "
        "(A1_SECONDARIES_FOR_PEDRA_MODEL*.csv)."
    )

# 2) Carregar e renomear colunas conhecidas
df_sec = read_csv_best_effort(src_sec)
rename_map = {c: NAME_MAP[c] for c in df_sec.columns if c in NAME_MAP}
df_sec = df_sec.rename(columns=rename_map)

if "timestamp" not in df_sec.columns:
    raise RuntimeError("A coluna 'timestamp' não foi encontrada após renomeação.")

# 3) Carregar referências (merge leve por timestamp, sem cálculos)
df_ref = None
if src_ref is not None:
    try:
        df_ref = read_csv_best_effort(src_ref)
        ref_rename = {c: NAME_MAP[c] for c in df_ref.columns if c in NAME_MAP}
        if ref_rename:
            df_ref = df_ref.rename(columns=ref_rename)
    except Exception:
        df_ref = None

# 4) Replicar por zonas disponíveis, PRESERVANDO tau_* como colunas
zonas_presentes = []
if "tau_densa" in df_sec.columns:
    zonas_presentes.append("densa")
if "tau_diluida" in df_sec.columns:
    zonas_presentes.append("diluida")
if "tau_backpass" in df_sec.columns:
    zonas_presentes.append("backpass")

if zonas_presentes:
    partes = []
    for z in zonas_presentes:
        parte = df_sec.copy()
        parte["zona"] = z
        partes.append(parte)
    df_z = pd.concat(partes, axis=0, ignore_index=True)
else:
    df_z = df_sec.copy()
    df_z["zona"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 5) Componente ausente → marcador
df_z["componente"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 6) Merge leve com referências (adiciona colunas que não existirem ainda)
if df_ref is not None and "timestamp" in df_ref.columns:
    cols_to_merge = [c for c in df_ref.columns if c != "timestamp" and c not in df_z.columns]
    if cols_to_merge:
        df_z = df_z.merge(df_ref[["timestamp"] + cols_to_merge], on="timestamp", how="left")

# 7) Garantir todas as colunas do padrão (criando vazias quando necessário)
for col in PADRAO_COLS:
    if col not in df_z.columns:
        df_z[col] = pd.NA

# 8) Reordenar e salvar
df_final = df_z[PADRAO_COLS].copy()
df_final.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

print(f"[OK] Tabela corrigida salva em:\n{OUT_PATH}")
print(f"Linhas: {len(df_final):,} | Colunas: {len(df_final.columns)}")
print("Colunas com algum preenchimento detectado:")
print([c for c in PADRAO_COLS if df_final[c].notna().any()])


[OK] Tabela corrigida salva em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao.csv
Linhas: 35,271 | Colunas: 18
Colunas com algum preenchimento detectado:
['timestamp', 'zona', 'componente', 'flw_total_c_t_h', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'tau_densa', 'tau_diluida', 'tau_backpass', 'o2_excess_pct']


In [7]:
# ============================================================
# EXPORTAR TABELA PADRÃO PARA PARQUET (SEM CÁLCULOS)
# ============================================================
import os
import pandas as pd

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
IN_CSV   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao.csv")
OUT_DIR  = os.path.join(DST_BASE, r"data\derivadas")
OUT_PQ   = os.path.join(OUT_DIR, "tabela_wr_wm_padrao.parquet")

# Garantir diretório de saída
os.makedirs(OUT_DIR, exist_ok=True)

# Ler CSV (conteúdo já consolidado anteriormente)
df = pd.read_csv(IN_CSV, low_memory=False)

# Salvar em Parquet (engine pyarrow, se disponível)
df.to_parquet(OUT_PQ, index=False)

# Validação rápida
df_check = pd.read_parquet(OUT_PQ)
print("[OK] Parquet gerado:")
print(" caminho:", OUT_PQ)
print(" shape :", df_check.shape)
print(" colunas:", list(df_check.columns))


[OK] Parquet gerado:
 caminho: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao.parquet
 shape : (35271, 18)
 colunas: ['timestamp', 'zona', 'componente', 'flw_total_c_t_h', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'tau_densa', 'tau_diluida', 'tau_backpass', 'o2_excess_pct', 'Wr', 'Wm', 'Wr_ref', 'Wm_ref', 'Wr_idx', 'Wm_idx']


In [8]:
# ============================================================
# DIAGNÓSTICO DE DUPLICIDADES NO PARQUET
# ============================================================
import os
import pandas as pd

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
PQ_PATH  = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao.parquet")

OUT_DIR  = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_DUP_ALL = os.path.join(OUT_DIR, "duplicatas_exatas.csv")
OUT_DUP_KEY = os.path.join(OUT_DIR, "duplicatas_por_chave_timestamp_zona_componente.csv")

# 1) Ler parquet
df = pd.read_parquet(PQ_PATH)
total = len(df)
print(f"[INFO] shape: {df.shape} (linhas={total}, colunas={df.shape[1]})")

# 2) Duplicatas EXATAS (todas as colunas iguais)
mask_dups_all = df.duplicated(keep=False)  # marca todos os membros de cada grupo duplicado
qtd_linhas_em_grupos_duplicados = int(mask_dups_all.sum())
qtd_linhas_duplicadas_alem_da_primeira = int(df.duplicated().sum())
qtd_unicas = int(len(df.drop_duplicates()))

print("\n[EXATAS]")
print(f" - Linhas pertencentes a grupos duplicados (inclui primeira ocorrência): {qtd_linhas_em_grupos_duplicados}")
print(f" - Linhas duplicadas além da primeira (contagem de 'extras'):          {qtd_linhas_duplicadas_alem_da_primeira}")
print(f" - Linhas únicas após remover exatas:                                 {qtd_unicas}")

# Exporta amostra das duplicatas exatas (se existir)
if qtd_linhas_em_grupos_duplicados > 0:
    df_dups_all = df[mask_dups_all].copy()
    # Para reduzir tamanho do arquivo, limitamos a 50k linhas na exportação (ajuste se quiser)
    if len(df_dups_all) > 50000:
        df_dups_all = df_dups_all.head(50000)
    df_dups_all.to_csv(OUT_DUP_ALL, index=False, encoding="utf-8-sig")
    print(f" - Inventário (amostra) salvo em: {OUT_DUP_ALL}")
else:
    print(" - Nenhuma duplicata exata encontrada.")

# 3) Duplicatas por CHAVE ['timestamp','zona','componente']
key_cols = [c for c in ["timestamp","zona","componente"] if c in df.columns]
if len(key_cols) == 3:
    grp = df.groupby(key_cols, dropna=False).size().reset_index(name="count")
    dup_keys = grp[grp["count"] > 1].sort_values("count", ascending=False)
    qtd_chaves_duplicadas = int(len(dup_keys))
    total_linhas_em_chaves_duplicadas = int(dup_keys["count"].sum())

    print("\n[POR CHAVE: timestamp, zona, componente]")
    print(f" - Quantidade de chaves duplicadas: {qtd_chaves_duplicadas}")
    print(f" - Total de linhas cobertas por essas chaves: {total_linhas_em_chaves_duplicadas}")

    if qtd_chaves_duplicadas > 0:
        # Exportar todas as chaves duplicadas
        dup_keys.to_csv(OUT_DUP_KEY, index=False, encoding="utf-8-sig")
        print(f" - Inventário de chaves duplicadas salvo em: {OUT_DUP_KEY}")
else:
    print("\n[POR CHAVE]")
    print(" - A checagem por chave foi pulada porque faltam colunas em ['timestamp','zona','componente'].")

# 4) Contagem por zona (útil para entender multiplicação esperada)
if "zona" in df.columns:
    print("\n[CONTAGEM POR ZONA]")
    print(df["zona"].value_counts(dropna=False).to_string())


[INFO] shape: (35271, 18) (linhas=35271, colunas=18)

[EXATAS]
 - Linhas pertencentes a grupos duplicados (inclui primeira ocorrência): 0
 - Linhas duplicadas além da primeira (contagem de 'extras'):          0
 - Linhas únicas após remover exatas:                                 35271
 - Nenhuma duplicata exata encontrada.

[POR CHAVE: timestamp, zona, componente]
 - Quantidade de chaves duplicadas: 0
 - Total de linhas cobertas por essas chaves: 0

[CONTAGEM POR ZONA]
zona
densa       11757
diluida     11757
backpass    11757


In [9]:
# ============================================================
# COLAPSAR ZONAS -> 1 LINHA POR TIMESTAMP (SEM CÁLCULOS)
# ============================================================
import os
import pandas as pd

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
IN_PQ    = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao.parquet")

OUT_DIR  = os.path.join(DST_BASE, r"data\derivadas")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PQ   = os.path.join(OUT_DIR, "tabela_wr_wm_padrao_dedup.parquet")
OUT_CSV  = os.path.join(OUT_DIR, "tabela_wr_wm_padrao_dedup.csv")

DIAG_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(DIAG_DIR, exist_ok=True)
OUT_CONFLITOS = os.path.join(DIAG_DIR, "conflitos_por_timestamp.csv")

# 1) Ler parquet atual
df = pd.read_parquet(IN_PQ)
print("[INFO] shape original:", df.shape)

# 2) Verificar conflitos entre as 3 linhas por timestamp (excluindo 'zona')
cols_check = [c for c in df.columns if c != "zona"]
conf_linhas = []

# (Opcional) primeiro, confirmar a multiplicidade por timestamp
# (não bloqueia se não for sempre 3)
mult_por_ts = df.groupby("timestamp", dropna=False).size()

# Varredura de conflitos (se colunas variam dentro do mesmo timestamp)
for ts, grp in df.groupby("timestamp", dropna=False):
    diffs = [c for c in cols_check if grp[c].nunique(dropna=False) > 1]
    if diffs:
        conf_linhas.append({
            "timestamp": ts,
            "n_linhas_grupo": len(grp),
            "colunas_com_diferencas": ", ".join(diffs)
        })

# Exporta conflitos, se existirem
if conf_linhas:
    pd.DataFrame(conf_linhas).to_csv(OUT_CONFLITOS, index=False, encoding="utf-8-sig")
    print(f"[ATENÇÃO] Conflitos detectados por timestamp. Inventário salvo em:\n{OUT_CONFLITOS}")
else:
    print("[INFO] Nenhum conflito detectado entre as 3 linhas por timestamp (além de 'zona').")

# 3) Colapsar: remover 'zona' e deduplicar por 'timestamp' (mantendo a primeira ocorrência)
df_nz = df.drop(columns=["zona"], errors="ignore").copy()

# Ordena por timestamp para garantir determinismo na escolha da primeira
df_nz = df_nz.sort_values(by=["timestamp"]).copy()

# Remove duplicatas por timestamp
df_dedup = df_nz.drop_duplicates(subset=["timestamp"], keep="first").copy()

# 4) Salvar resultados
df_dedup.to_parquet(OUT_PQ, index=False)
df_dedup.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

print("\n[OK] Tabela deduplicada gerada:")
print(" - Parquet:", OUT_PQ)
print(" - CSV    :", OUT_CSV)
print("Shape final:", df_dedup.shape)

# 5) Validações auxiliares
if "timestamp" in df_dedup.columns:
    n_ts_unique = df_dedup["timestamp"].nunique(dropna=False)
    print("Total de timestamps únicos:", n_ts_unique)
else:
    print("[ALERTA] Coluna 'timestamp' ausente após deduplicação (não esperado).")


[INFO] shape original: (35271, 18)
[INFO] Nenhum conflito detectado entre as 3 linhas por timestamp (além de 'zona').

[OK] Tabela deduplicada gerada:
 - Parquet: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao_dedup.parquet
 - CSV    : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao_dedup.csv
Shape final: (11757, 17)
Total de timestamps únicos: 11757


In [11]:
# ============================================================
# MVP DE MODELO: ISOLATION FOREST (CORREÇÃO NumPy 2.0)
# ============================================================
import os
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import Pipeline
import joblib

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ    = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao_dedup.parquet")
DERIV_DIR = os.path.join(DST_BASE, r"data\derivadas")
MODEL_DIR = os.path.join(DST_BASE, r"outputs\modelos")
DIAG_DIR  = os.path.join(DST_BASE, r"outputs\diagnosticos")

os.makedirs(DERIV_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(DIAG_DIR, exist_ok=True)

# 1) Ler base deduplicada
df = pd.read_parquet(IN_PQ)

# 2) Seleção de features já preenchidas
features = [
    "flw_total_c_t_h",
    "total_air_flow_knm3_h",
    "total_paf_air_flow_knm3_h",
    "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy",
    "tau_densa",
    "tau_diluida",
    "tau_backpass",
    "o2_excess_pct",
]
features = [c for c in features if c in df.columns]

# 3) Subconjunto de dados (timestamp + features)
df_model = df[["timestamp"] + features].copy()
for c in features:
    df_model[c] = pd.to_numeric(df_model[c], errors="coerce")

# 4) Pipeline: Imputer (mediana) + StandardScaler + IsolationForest
pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("iso", IsolationForest(
        n_estimators=300,
        max_samples="auto",
        contamination=0.03,   # ~3% anomalias
        random_state=42,
        n_jobs=-1
    ))
])

X = df_model[features].values
pipeline.fit(X)

# 5) Scores e rótulos (usando o pipeline para aplicar transformações)
# decision_function: maior = mais normal. Vamos inverter para anomalia (maior = mais anômalo).
raw_scores = pipeline.decision_function(X)  # array shape (n,)
anom_base = -raw_scores
rng = np.ptp(anom_base)  # max - min (NumPy 2.0+)
if not np.isfinite(rng) or rng == 0:
    rng = 1e-12
anomalia_score = (anom_base - np.min(anom_base)) / rng  # 0..1

pred_labels = pipeline.predict(X)  # -1 anomalia, 1 normal
is_anomalia = (pred_labels == -1).astype(int)

# 6) Montar saída
out = df_model.copy()
out["anomalia_score"] = anomalia_score
out["is_anomalia"] = is_anomalia

# 7) Salvar derivados
out_pq  = os.path.join(DERIV_DIR, "anomalias_isoforest.parquet")
out_csv = os.path.join(DERIV_DIR, "anomalias_isoforest.csv")
out.to_parquet(out_pq, index=False)
out.to_csv(out_csv, index=False, encoding="utf-8-sig")

# 8) Salvar modelo
model_path = os.path.join(MODEL_DIR, "isoforest_baseline.pkl")
joblib.dump(pipeline, model_path)

# 9) Estatísticas simples
pct_anom = 100.0 * out["is_anomalia"].mean()
sumario = []
sumario.append(f"Registros: {len(out):,}")
sumario.append(f"Features usadas: {features}")
sumario.append(f"% anomalias (contamination alvo=3%): {pct_anom:.2f}%")
top5 = out.sort_values("anomalia_score", ascending=False).head(5)[["timestamp","anomalia_score"]]
sumario.append("\nTop 5 timestamps mais anômalos (maior score):")
sumario.append(top5.to_string(index=False))

diag_path = os.path.join(DIAG_DIR, "anomalia_stats.txt")
with open(diag_path, "w", encoding="utf-8") as f:
    f.write("\n".join(sumario))

print("[OK] Derivados salvos:")
print(" -", out_pq)
print(" -", out_csv)
print("[OK] Modelo salvo em:", model_path)
print("\nResumo:")
print("\n".join(sumario))


[OK] Derivados salvos:
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\anomalias_isoforest.parquet
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\anomalias_isoforest.csv
[OK] Modelo salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\modelos\isoforest_baseline.pkl

Resumo:
Registros: 11,757
Features usadas: ['flw_total_c_t_h', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'tau_densa', 'tau_diluida', 'tau_backpass', 'o2_excess_pct']
% anomalias (contamination alvo=3%): 3.00%

Top 5 timestamps mais anômalos (maior score):
          timestamp  anomalia_score
2023-07-04 04:00:00        1.000000
2023-05-31 11:00:00        0.996667
2023-11-16 19:00:00        0.990161
2023-05-31 09:00:00        0.954710
2023-10-11 21:00:00        0.926770


In [12]:
# ============================================================
# PHYSICS-BASED (MVP): Wr/Wm + refs + índices
# ============================================================
import os
import json
import numpy as np
import pandas as pd

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao_dedup.parquet")
REFS_CSV= os.path.join(SRC_BASE, r"outputs\pedra\A1_SECONDARIES_REFS.csv")

OUT_DIR = os.path.join(DST_BASE, r"data\derivadas")
LOG_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

OUT_PQ  = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste.parquet")
OUT_CSV = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste.csv")
OUT_LOG = os.path.join(LOG_DIR, "physics_params_mvp.json")

# --- 1) Ler base deduplicada ---
df = pd.read_parquet(IN_PQ)

# Garantir numérico nas colunas usadas
def to_num(s):
    return pd.to_numeric(s, errors="coerce")

for c in ["delta_proxy","tau_densa","tau_diluida","tau_backpass",
          "total_air_flow_knm3_h","total_paf_air_flow_knm3_h","flw_total_c_t_h"]:
    if c in df.columns:
        df[c] = to_num(df[c])

# --- 2) Ler referências (se houver) ---
def read_csv_best_effort(path):
    for enc in ("utf-8-sig","latin1"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return None

refs = read_csv_best_effort(REFS_CSV)

# Extrair refs como únicos valores (esperado 1 linha). Se houver várias, pegar a 1ª não nula.
def pick_ref(series):
    if series is None:
        return np.nan
    s = pd.to_numeric(series, errors="coerce")
    s = s.dropna()
    return s.iloc[0] if len(s) else np.nan

ref_vals = {}
if refs is not None:
    for col in ["delta_proxy_ref","tau_densa_ref","tau_diluida_ref","tau_backpass_ref",
                "v_proxy_total_ref","v_proxy_primary_ref"]:
        if col in refs.columns:
            ref_vals[col] = pick_ref(refs[col])
        else:
            ref_vals[col] = np.nan
else:
    # sem arquivo de refs
    ref_vals = {k: np.nan for k in
        ["delta_proxy_ref","tau_densa_ref","tau_diluida_ref","tau_backpass_ref","v_proxy_total_ref","v_proxy_primary_ref"]}

# --- 3) Definir variáveis de trabalho (MVP) ---
# τ (°C) -> τK (K). Usaremos média das três zonas disponíveis.
taus = [c for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df.columns]
df["tau_mean_C"] = np.nan
if taus:
    df["tau_mean_C"] = pd.concat([df[t] for t in taus], axis=1).mean(axis=1, skipna=True)

# v (proxy de velocidade): preferimos ar total; se não tiver, ar primário; fallback: mediana.
if "total_air_flow_knm3_h" in df.columns:
    v_curr = df["total_air_flow_knm3_h"].copy()
elif "total_paf_air_flow_knm3_h" in df.columns:
    v_curr = df["total_paf_air_flow_knm3_h"].copy()
else:
    v_curr = pd.Series(np.nan, index=df.index)

# delta_proxy atual
delta_curr = df["delta_proxy"] if "delta_proxy" in df.columns else pd.Series(np.nan, index=df.index)

# Refs: se existirem no CSV, usamos; senão, fallback pela mediana dos próprios dados
v_ref = ref_vals.get("v_proxy_total_ref", np.nan)
if not np.isfinite(v_ref):
    v_ref = np.nanmedian(v_curr.values)

delta_ref = ref_vals.get("delta_proxy_ref", np.nan)
if not np.isfinite(delta_ref):
    delta_ref = np.nanmedian(delta_curr.values)

tau_ref_C_vals = [ref_vals.get("tau_densa_ref", np.nan),
                  ref_vals.get("tau_diluida_ref", np.nan),
                  ref_vals.get("tau_backpass_ref", np.nan)]
tau_ref_C = np.nanmean([x for x in tau_ref_C_vals if np.isfinite(x)]) if np.any(np.isfinite(tau_ref_C_vals)) else np.nan
if not np.isfinite(tau_ref_C):
    tau_ref_C = np.nanmedian(df["tau_mean_C"].values)

# --- 4) Constantes do MVP (ajustáveis) ---
params = {
    "alpha": 1.0,        # fatores escala (cancelam nos índices)
    "beta":  1.0,
    "n_r":   2.0,        # expoente de v para Wr
    "m_r":   1.0,        # expoente de delta para Wr
    "n_m":   3.0,        # expoente de v para Wm
    "m_m":   0.5,        # expoente de delta para Wm
    "Tcrit": 1200.0,     # K (refratário)
    "Topt":  1250.0      # K (metálico)
}

# --- 5) Cálculo (Wr/Wm e refs) ---
# Preparos
tauK     = to_num(df["tau_mean_C"]) + 273.15
tau_refK = float(tau_ref_C) + 273.15

v_curr_v = to_num(v_curr).values
delta_v  = to_num(delta_curr).values

# Wr e Wm atuais (proxy adimensional de desgaste)
Wr = params["alpha"] * np.power(v_curr_v, params["n_r"]) * np.power(delta_v, params["m_r"]) * np.exp(-tauK.values / params["Tcrit"])
Wm = params["beta"]  * np.power(v_curr_v, params["n_m"]) * np.power(delta_v, params["m_m"]) * np.exp(-tauK.values / params["Topt"])

# Wr_ref e Wm_ref (constantes, usando refs)
Wr_ref = params["alpha"] * (v_ref ** params["n_r"]) * (delta_ref ** params["m_r"]) * np.exp(-tau_refK / params["Tcrit"])
Wm_ref = params["beta"]  * (v_ref ** params["n_m"]) * (delta_ref ** params["m_m"]) * np.exp(-tau_refK / params["Topt"])

# Índices (cuidados para divisão por zero/NaN)
def safe_div(a, b):
    out = np.full_like(a, np.nan, dtype="float64")
    if b is not None and np.isfinite(b) and b != 0:
        out = a / b
    return out

Wr_idx = safe_div(Wr, Wr_ref)
Wm_idx = safe_div(Wm, Wm_ref)

# --- 6) Gravar na tabela final (não sobrescreve a original) ---
df_out = df.copy()
df_out["Wr"] = Wr
df_out["Wm"] = Wm
df_out["Wr_ref"] = Wr_ref if np.isfinite(Wr_ref) else np.nan
df_out["Wm_ref"] = Wm_ref if np.isfinite(Wm_ref) else np.nan
df_out["Wr_idx"] = Wr_idx
df_out["Wm_idx"] = Wm_idx

df_out.to_parquet(OUT_PQ, index=False)
df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# --- 7) Log de parâmetros e referências usadas ---
log = {
    "refs_arquivo_encontrado": os.path.exists(REFS_CSV),
    "refs_utilizadas": ref_vals,
    "fallbacks": {
        "v_ref_usou_median?"     : not np.isfinite(ref_vals.get("v_proxy_total_ref", np.nan)),
        "delta_ref_usou_median?" : not np.isfinite(ref_vals.get("delta_proxy_ref", np.nan)),
        "tau_ref_usou_median?"   : not np.any(np.isfinite(tau_ref_C_vals)),
    },
    "params": params,
    "coluna_v_atual": "total_air_flow_knm3_h" if "total_air_flow_knm3_h" in df.columns
                      else ("total_paf_air_flow_knm3_h" if "total_paf_air_flow_knm3_h" in df.columns else None),
    "linhas": len(df_out),
    "colunas": list(df_out.columns)
}
with open(OUT_LOG, "w", encoding="utf-8") as f:
    json.dump(log, f, ensure_ascii=False, indent=2)

# --- 8) Resumo em tela ---
print("[OK] Tabela com desgaste salva:")
print(" -", OUT_PQ)
print(" -", OUT_CSV)
print("\nResumo rápido:")
print("  Wr_ref:", Wr_ref, " | Wm_ref:", Wm_ref)
print("  % NaN Wr_idx:", np.mean(~np.isfinite(Wr_idx)) * 100, "% | % NaN Wm_idx:", np.mean(~np.isfinite(Wm_idx)) * 100, "%")
print("\nTop 5 maiores Wr_idx:")
print(pd.DataFrame({"timestamp": df_out["timestamp"], "Wr_idx": Wr_idx}).sort_values("Wr_idx", ascending=False).head(5).to_string(index=False))
print("\nTop 5 maiores Wm_idx:")
print(pd.DataFrame({"timestamp": df_out["timestamp"], "Wm_idx": Wm_idx}).sort_values("Wm_idx", ascending=False).head(5).to_string(index=False))
print("\n[INFO] Parâmetros e fallbacks registrados em:", OUT_LOG)


[OK] Tabela com desgaste salva:
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste.parquet
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste.csv

Resumo rápido:
  Wr_ref: 166056.27285204153  | Wm_ref: 248106045.49944544
  % NaN Wr_idx: 36.437866802755806 % | % NaN Wm_idx: 81.20268776048312 %

Top 5 maiores Wr_idx:
          timestamp    Wr_idx
2024-09-29 14:00:00 12.490335
2024-03-29 05:00:00 11.579226
2024-03-29 16:00:00 11.470146
2024-03-29 02:00:00 11.241728
2024-03-29 01:00:00 10.761733

Top 5 maiores Wm_idx:
          timestamp   Wm_idx
2024-09-29 14:00:00 3.428456
2024-03-29 05:00:00 3.291170
2024-03-29 16:00:00 3.263653
2024-03-29 02:00:00 3.181821
2024-03-29 01:00:00 3.140810

[INFO] Parâmetros e fallbacks registrados em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\physics_params_mvp.json


C:\Users\wilso\AppData\Local\Temp\ipykernel_26704\1751241561.py:124: RuntimeWarning: invalid value encountered in power
  Wm = params["beta"]  * np.power(v_curr_v, params["n_m"]) * np.power(delta_v, params["m_m"]) * np.exp(-tauK.values / params["Topt"])


In [13]:
# ============================================================
# GRÁFICOS TEMPORAIS DE Wr_idx / Wm_idx + MARCAÇÃO DE BASELINE
# ============================================================
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste.parquet")
LOG_JSON= os.path.join(DST_BASE, r"outputs\diagnosticos\physics_params_mvp.json")

# possíveis fontes de período-baseline (opcionais)
CANDIDATES_BASELINE = [
    os.path.join(SRC_BASE, r"outputs\baseline_datasets\physics_baseline_proxies.csv"),
    os.path.join(SRC_BASE, r"outputs\baseline_mask.csv"),
]

OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Ler tabela com desgaste
df = pd.read_parquet(IN_PQ)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp")

# 2) Carregar log para informar se refs vieram de medianas
fallback_info = {}
if os.path.exists(LOG_JSON):
    with open(LOG_JSON, "r", encoding="utf-8") as f:
        log_data = json.load(f)
    fallback_info = log_data.get("fallbacks", {})

# 3) Descobrir período de baseline (se existir)
baseline_range = None
baseline_source = None

# tenta arquivo de proxies de baseline (min..max do timestamp)
bp = CANDIDATES_BASELINE[0]
if os.path.exists(bp):
    try:
        bdf = pd.read_csv(bp, encoding="utf-8-sig")
    except Exception:
        bdf = pd.read_csv(bp, encoding="latin1")
    if "timestamp" in bdf.columns:
        bdf["timestamp"] = pd.to_datetime(bdf["timestamp"], errors="coerce")
        bdf = bdf.dropna(subset=["timestamp"])
        if not bdf.empty:
            baseline_range = (bdf["timestamp"].min(), bdf["timestamp"].max())
            baseline_source = os.path.relpath(bp, SRC_BASE)

# tenta máscara booleana
if baseline_range is None:
    bm = CANDIDATES_BASELINE[1]
    if os.path.exists(bm):
        try:
            mdf = pd.read_csv(bm, encoding="utf-8-sig")
        except Exception:
            mdf = pd.read_csv(bm, encoding="latin1")
        if {"timestamp","is_baseline"}.issubset(mdf.columns):
            mdf["timestamp"] = pd.to_datetime(mdf["timestamp"], errors="coerce")
            mdf = mdf[mdf["is_baseline"]==1].dropna(subset=["timestamp"])
            if not mdf.empty:
                baseline_range = (mdf["timestamp"].min(), mdf["timestamp"].max())
                baseline_source = os.path.relpath(bm, SRC_BASE)

# 4) Função helper para plotar série e sombrear baseline
def plot_idx(df, col_idx, out_png, title):
    s = df[["timestamp", col_idx]].dropna().copy()
    if s.empty:
        print(f"[INFO] Não há dados válidos para {col_idx}. Gráfico não gerado.")
        return

    plt.figure(figsize=(12,4))
    plt.plot(s["timestamp"], s[col_idx])
    plt.axhline(1.0, linestyle="--")  # linha do índice 1.0

    if baseline_range is not None:
        plt.axvspan(baseline_range[0], baseline_range[1], alpha=0.15)
        subtitle = f"Baseline sombreado (fonte: {baseline_source})"
    else:
        # informar na figura que não há período explícito
        # (refs podem ter vindo de constantes/medianas)
        fb_txt = []
        for k,v in fallback_info.items():
            if isinstance(v, bool):
                fb_txt.append(f"{k}={'sim' if v else 'não'}")
        subtitle = "Sem período de baseline localizado. " + (", ".join(fb_txt) if fb_txt else "")

    plt.title(f"{title}\n{subtitle}")
    plt.xlabel("timestamp")
    plt.ylabel(col_idx)
    plt.tight_layout()
    plt.savefig(out_png, dpi=120)
    plt.close()
    print("[OK] Gráfico salvo em:", out_png)

# 5) Gerar gráficos
plot_idx(df, "Wr_idx", os.path.join(OUT_DIR, "wr_idx_timeline.png"), "Wr_idx ao longo do tempo")
plot_idx(df, "Wm_idx", os.path.join(OUT_DIR, "wm_idx_timeline.png"), "Wm_idx ao longo do tempo")

# 6) Feedback em texto (terminal)
if baseline_range is not None:
    print(f"[INFO] Período de baseline usado para sombreamento: {baseline_range[0]} → {baseline_range[1]}")
else:
    print("[INFO] Não foi encontrado período explícito de baseline nos artefatos padrão.")


[OK] Gráfico salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_idx_timeline.png
[OK] Gráfico salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wm_idx_timeline.png
[INFO] Período de baseline usado para sombreamento: 2024-03-04 20:00:00 → 2024-03-12 15:00:00
